# L2D Showcase

The following code is a modified version of [test_learned_on_benchmark.py](l2d/test_learned_on_benchmark.py)

In [1]:
import sys
sys.path.append("l2d")
import os
os.chdir("l2d")

In [2]:
from mb_agg import *
from agent_utils import *
from Params import configs
from l2d.JSSP_Env import SJSSP
from l2d.PPO_jssp_multiInstances import PPO

import time
import torch
import argparse
import numpy as np

In [3]:
def parse_instance_taillard(filename):
    '''Parses instance written in Taillard specification: http://jobshop.jjvh.nl/explanation.php
    
      Args:
        filename - file containing the instance in Taillard specification

      Returns:
        number of jobs,
        number of machines,
        the processor times for each operation,
        the order for visiting the machines
    '''

    with open(filename, 'r') as f:
        # parse number of jobs J and machines M
        J, M = map(int, f.readline().split())

        # Initialize two empty numpy arrays with dimensions J x M
        processor_times = np.empty((J, M), dtype=int)
        orders_of_machines = np.empty((J, M), dtype=int)
    
        # Read the next J lines containing processor times
        for i in range(J):
            processor_times[i] = list(map(int, f.readline().split()))
    
        # Read the next J lines containing orders of machines
        for i in range(J):
            orders_of_machines[i] = list(map(int, f.readline().split()))

        return J, M, processor_times, orders_of_machines

In [4]:
def solve_instance(instance: str, model: str, device='cpu'):
    '''Solves instance using given model and prints out the makespan
    
      Args:
          instance - file with the instance in Taillard specification
          model - network model to use as an agent

      Returns:
          makespan
    '''

    # parse the instance
    jobs, machines, times, orders = parse_instance_taillard(instance)

    # load agents environment
    env = SJSSP(n_j=jobs, n_m=machines)
    adj, fea, candidate, mask = env.reset((times, orders))
    ep_reward = - env.max_endTime

    # load the agent
    ppo = PPO(configs.lr, configs.gamma, configs.k_epochs, configs.eps_clip,
              n_j=jobs,
              n_m=machines,
              num_layers=configs.num_layers,
              neighbor_pooling_type=configs.neighbor_pooling_type,
              input_dim=configs.input_dim,
              hidden_dim=configs.hidden_dim,
              num_mlp_layers_feature_extract=configs.num_mlp_layers_feature_extract,
              num_mlp_layers_actor=configs.num_mlp_layers_actor,
              hidden_dim_actor=configs.hidden_dim_actor,
              num_mlp_layers_critic=configs.num_mlp_layers_critic,
              hidden_dim_critic=configs.hidden_dim_critic)
    ppo.policy.load_state_dict(torch.load(MODEL, map_location=torch.device('cpu')))
    g_pool_step = g_pool_cal(graph_pool_type=configs.graph_pool_type,
                             batch_size=torch.Size([1, env.number_of_tasks, env.number_of_tasks]),
                             n_nodes=env.number_of_tasks,
                             device=device)

    # run the experiment
    while True:
        fea_tensor = torch.from_numpy(np.copy(fea)).to(device)
        adj_tensor = torch.from_numpy(np.copy(adj)).to(device).to_sparse()
        candidate_tensor = torch.from_numpy(np.copy(candidate)).to(device)
        mask_tensor = torch.from_numpy(np.copy(mask)).to(device)
    
        with torch.no_grad():
            pi, _ = ppo.policy(x=fea_tensor,
                               graph_pool=g_pool_step,
                               padded_nei=None,
                               adj=adj_tensor,
                               candidate=candidate_tensor.unsqueeze(0),
                               mask=mask_tensor.unsqueeze(0))
            action = greedy_select_action(pi, candidate)
    
        adj, fea, reward, done, candidate, mask = env.step(action)
        ep_reward += reward
    
        if done:
            break
            
    return env.posRewards - ep_reward

# 30x20 Taillard's instances
I test the "30x20" model on Taillard's 30x20 instances.

In [6]:
MODEL = "SavedNetwork/30_20_1_99.pth"
BENCHMARKS_PATH = "../../../benchmarks/ta_instances/Taillard_specification/"
INSTANCES = [
    'ta41.txt',
    'ta42.txt',
    'ta43.txt',
    'ta44.txt',
    'ta45.txt',
    'ta46.txt',
    'ta47.txt',
    'ta48.txt',
    'ta49.txt',
    'ta50.txt',
]

for instance in INSTANCES:
    makespan = solve_instance(BENCHMARKS_PATH + instance, MODEL)
    print(f"Makespan of instance '{instance}': {makespan}")

/Users/marosbratko/Github/Graph-neural-networks-and-deep-reinforcement-learning-in-job-scheduling/models/L2D/l2d/mb_agg.py:44: UserWarning: torch.sparse.SparseTensor(indices, values, shape, *, device=) is deprecated.  Please use torch.sparse_coo_tensor(indices, values, shape, dtype=, device=). (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:607.)
  graph_pool = torch.sparse.FloatTensor(idx, elem,


Makespan of instance 'ta41.txt': 2592.0
Makespan of instance 'ta42.txt': 2715.0
Makespan of instance 'ta43.txt': 2431.0
Makespan of instance 'ta44.txt': 2712.0
Makespan of instance 'ta45.txt': 2651.0
Makespan of instance 'ta46.txt': 2852.0
Makespan of instance 'ta47.txt': 2502.0
Makespan of instance 'ta48.txt': 2525.0
Makespan of instance 'ta49.txt': 2497.0
Makespan of instance 'ta50.txt': 2507.0


# All benchmarks

I test the "30x20" model on all available instances in Taillard specification.

In [7]:
def get_all_instances_in_taillard_specification():
    '''Lists all instances in Taillard specification'''
    matching_files = []
    root_dir = "../../../benchmarks/"
    target_string = "Taillard_specification"

    for foldername, subfolders, filenames in os.walk(root_dir):
        for filename in filenames:
            filepath = os.path.join(foldername, filename)
            if target_string in filepath:
                matching_files.append(filepath)

    return matching_files

In [9]:
MODEL = "SavedNetwork/30_20_1_99.pth"
instances = get_all_instances_in_taillard_specification()
for instance in instances:
    makespan = solve_instance(instance, MODEL)
    print(f"Makespan of instance '{instance.split('/')[-1]}': {makespan}")

Makespan of instance 'dmu36.txt': 7308.0
Makespan of instance 'dmu22.txt': 5606.0
Makespan of instance 'dmu23.txt': 5267.0
Makespan of instance 'dmu37.txt': 7364.0
Makespan of instance 'dmu21.txt': 5706.0
Makespan of instance 'dmu35.txt': 6457.0
Makespan of instance 'dmu09.txt': 4377.0
Makespan of instance 'dmu08.txt': 4071.0
Makespan of instance 'dmu34.txt': 6514.0
Makespan of instance 'dmu20.txt': 4715.0
Makespan of instance 'dmu18.txt': 4908.0
Makespan of instance 'dmu24.txt': 5626.0
Makespan of instance 'dmu30.txt': 6130.0
Makespan of instance 'dmu31.txt': 6971.0
Makespan of instance 'dmu25.txt': 4945.0
Makespan of instance 'dmu19.txt': 4862.0
Makespan of instance 'dmu33.txt': 6155.0
Makespan of instance 'dmu27.txt': 6140.0
Makespan of instance 'dmu26.txt': 6089.0
Makespan of instance 'dmu32.txt': 6572.0
Makespan of instance 'dmu55.txt': 6582.0
Makespan of instance 'dmu41.txt': 4738.0
Makespan of instance 'dmu69.txt': 8729.0
Makespan of instance 'dmu68.txt': 8713.0
Makespan of inst